In [24]:
import numpy as np
import pandas as pd
#import scikit-learn
import os
import torch
from transformers import BertForSequenceClassification
import torch.nn as nn
import torch.optim as optim
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import mlflow
from mlflow import log_metric, log_param, log_params, log_artifacts
#import tensorflow
#from tensorflow.summary import create_file_writer

In [4]:
from transformers import BertForSequenceClassification

In [5]:
DATA_PATH='/home/vboxuser/mlprojects/data/made_with_ml'
MODEL_NAME = 'bert-base-nli-mean-tokens'

In [6]:
def read_and_process_data(parent_dir,filename):
    '''
    This function will read and prepare text for training
    '''
    df=pd.read_csv(os.path.join(parent_dir,filename),dtype={'title':str,'description':str,'tag':str},index_col=None)
    df['combined']=df['title'] + ' ' + df['description']
    df['combined']=df['combined'].apply(lambda text:text.lower())
    df['combined_with_SEP']=df['title'] + ' [SEP] ' + df['description']
    df['combined_with_SEP']=df['combined_with_SEP'].apply(lambda text:text.lower())
    return df

In [7]:
df_train=read_and_process_data(DATA_PATH,'dataset.csv')
df_valid=read_and_process_data(DATA_PATH,'holdout.csv')

In [8]:
df_train.head()

,id,created_on,title,description,tag,combined,combined_with_SEP
0,6,2020-02-20 06:43:18,Comparison between YOLO and RCNN on real world...,Bringing theory to experiment is cool. We can ...,computer-vision,comparison between yolo and rcnn on real world...,comparison between yolo and rcnn on real world...
1,7,2020-02-20 06:47:21,"Show, Infer & Tell: Contextual Inference for C...",The beauty of the work lies in the way it arch...,computer-vision,"show, infer & tell: contextual inference for c...","show, infer & tell: contextual inference for c..."
2,9,2020-02-24 16:24:45,Awesome Graph Classification,"A collection of important graph embedding, cla...",graph-learning,awesome graph classification a collection of i...,awesome graph classification [sep] a collectio...
3,15,2020-02-28 23:55:26,Awesome Monte Carlo Tree Search,A curated list of Monte Carlo tree search pape...,reinforcement-learning,awesome monte carlo tree search a curated list...,awesome monte carlo tree search [sep] a curate...
4,25,2020-03-07 23:04:31,AttentionWalk,"A PyTorch Implementation of ""Watch Your Step: ...",graph-learning,"attentionwalk a pytorch implementation of ""wat...",attentionwalk [sep] a pytorch implementation o...


In [9]:
df_train['tag'].unique()

array(['computer-vision', 'graph-learning', 'reinforcement-learning',
       'natural-language-processing', 'mlops', 'time-series'],
      dtype=object)

In [10]:
def model_init(model_name='sentence-transformers/all-mpnet-base-v2'):
    model = SentenceTransformer(model_name)
    return model

In [11]:
def featurization(text,model):
    embeddings = model.encode(text)
    return embeddings

In [ ]:
def label_encoding(self):
    le = preprocessing.LabelEncoder()
    le.fit(self.df['tag'])
    return le

def label_transform(df,le):
    return le.transform(df['tag'])

In [13]:
model=model_init(model_name='sentence-transformers/all-mpnet-base-v2')
df_train['bert_features_combined']=df_train['combined'].apply(lambda x: featurization(x,model))
df_train['bert_features_combined_SEP']=df_train['combined_with_SEP'].apply(lambda x: featurization(x,model))

In [14]:
df_valid['bert_features_combined']=df_valid['combined'].apply(lambda x: featurization(x,model))
df_valid['bert_features_combined_SEP']=df_valid['combined_with_SEP'].apply(lambda x: featurization(x,model))

In [15]:
label_encoder = LabelEncoder()
label_encoder.fit(df_train['tag'])
df_train['label_int'] = label_encoder.transform(df_train['tag'])
df_valid['label_int'] = label_encoder.transform(df_valid['tag'])

In [16]:
le = OneHotEncoder(handle_unknown='ignore')
#le.fit(df_train['label_int'])
le.fit(df_train['label_int'].values.reshape(-1, 1))

OneHotEncoder(handle_unknown='ignore')

In [17]:
label_int_train = df_train['label_int'].values.reshape(-1, 1)
label_int_valid = df_valid['label_int'].values.reshape(-1, 1)

# Create and fit OneHotEncoder
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoder.fit(label_int_train)

# Transform and convert to DataFrame
y_train = torch.tensor(onehot_encoder.transform(label_int_train).toarray())
y_valid = torch.tensor(onehot_encoder.transform(label_int_valid).toarray())

X_train=torch.tensor(df_train['bert_features_combined'])
X_valid=torch.tensor(df_valid['bert_features_combined'])


/tmp/ipykernel_13866/1907311675.py:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /croot/pytorch_1686931851744/work/torch/csrc/utils/tensor_new.cpp:245.)
  X_train=torch.tensor(df_train['bert_features_combined'])


In [18]:
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

valid_dataset = TensorDataset(X_valid, y_valid)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [19]:
#log_dir = "logs"  # Specify the log directory path
#writer = create_file_writer(log_dir)
writer = SummaryWriter()

In [25]:
mlflow.set_experiment("topic_classification")

2023/08/16 16:58:13 INFO mlflow.tracking.fluent: Experiment with name 'topic_classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/vboxuser/mlprojects/sample/notebooks/mlruns/728078586741920700', creation_time=1692219493226, experiment_id='728078586741920700', last_update_time=1692219493226, lifecycle_stage='active', name='topic_classification', tags={}>

In [26]:
with mlflow.start_run() as run:
    run_id = run.info.run_id

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [28]:
# Generate some example data (replace this with your actual data)
input_size = 768
num_classes = 6

# Define the neural network model
class TwoLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(TwoLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Initialize the model
hidden_size = 128
dropout_prob = 0.3
model = TwoLayerNN(input_size, hidden_size, num_classes, dropout_prob)

mlflow.log_param("hidden_size", hidden_size)
mlflow.log_param("learning_rate", 0.001)
mlflow.log_param('no of layers',2)
mlflow.log_param('dropout',0.3)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_X, batch_y in train_dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))
        loss.backward()
        optimizer.step()

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        valid_loss = 0.0
        for batch_X, batch_y in valid_dataloader:
            outputs = model(batch_X)
            loss = criterion(outputs, torch.argmax(batch_y, dim=1))
            valid_loss += loss.item()
    with torch.no_grad():
        train_loss = 0.0
        for batch_X, batch_y in train_dataloader:
            outputs = model(batch_X)
            loss = criterion(outputs, torch.argmax(batch_y, dim=1))
            train_loss += loss.item()

        avg_valid_loss = valid_loss / len(valid_dataloader)
        avg_train_loss = train_loss / len(train_dataloader)
        print(f"Epoch [{epoch+1}/{num_epochs}] - Training Loss: {avg_train_loss:.4f}")
        print(f"Epoch [{epoch+1}/{num_epochs}] - Validation Loss: {avg_valid_loss:.4f}")
        
    
    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar("Loss/valid", valid_loss, epoch)
    
mlflow.log_metric("train_loss", train_loss)
mlflow.log_metric("valid_loss", valid_loss)
writer.flush()
    

# Make predictions
with torch.no_grad():
    predictions = model(X_valid)
    predicted_classes = torch.argmax(predictions, dim=1)

# You can use the predicted_classes for further analysis or evaluation


Epoch [1/1000] - Training Loss: 1.6448
Epoch [1/1000] - Validation Loss: 1.6424
Epoch [2/1000] - Training Loss: 1.4017
Epoch [2/1000] - Validation Loss: 1.3928
Epoch [3/1000] - Training Loss: 1.3241
Epoch [3/1000] - Validation Loss: 1.3122
Epoch [4/1000] - Training Loss: 1.3029
Epoch [4/1000] - Validation Loss: 1.2918
Epoch [5/1000] - Training Loss: 1.2892
Epoch [5/1000] - Validation Loss: 1.2805
Epoch [6/1000] - Training Loss: 1.2623
Epoch [6/1000] - Validation Loss: 1.2537
Epoch [7/1000] - Training Loss: 1.2372
Epoch [7/1000] - Validation Loss: 1.2298
Epoch [8/1000] - Training Loss: 1.2231
Epoch [8/1000] - Validation Loss: 1.2168
Epoch [9/1000] - Training Loss: 1.2090
Epoch [9/1000] - Validation Loss: 1.2039
Epoch [10/1000] - Training Loss: 1.1908
Epoch [10/1000] - Validation Loss: 1.1850
Epoch [11/1000] - Training Loss: 1.1757
Epoch [11/1000] - Validation Loss: 1.1730
Epoch [12/1000] - Training Loss: 1.1612
Epoch [12/1000] - Validation Loss: 1.1639
Epoch [13/1000] - Training Loss: 1

Epoch [104/1000] - Training Loss: 1.0453
Epoch [104/1000] - Validation Loss: 1.0804
Epoch [105/1000] - Training Loss: 1.0453
Epoch [105/1000] - Validation Loss: 1.0805
Epoch [106/1000] - Training Loss: 1.0452
Epoch [106/1000] - Validation Loss: 1.0813
Epoch [107/1000] - Training Loss: 1.0452
Epoch [107/1000] - Validation Loss: 1.0813
Epoch [108/1000] - Training Loss: 1.0452
Epoch [108/1000] - Validation Loss: 1.0808
Epoch [109/1000] - Training Loss: 1.0452
Epoch [109/1000] - Validation Loss: 1.0809
Epoch [110/1000] - Training Loss: 1.0452
Epoch [110/1000] - Validation Loss: 1.0808
Epoch [111/1000] - Training Loss: 1.0452
Epoch [111/1000] - Validation Loss: 1.0806
Epoch [112/1000] - Training Loss: 1.0452
Epoch [112/1000] - Validation Loss: 1.0806
Epoch [113/1000] - Training Loss: 1.0452
Epoch [113/1000] - Validation Loss: 1.0808
Epoch [114/1000] - Training Loss: 1.0452
Epoch [114/1000] - Validation Loss: 1.0809
Epoch [115/1000] - Training Loss: 1.0452
Epoch [115/1000] - Validation Loss:

Epoch [209/1000] - Training Loss: 1.0436
Epoch [209/1000] - Validation Loss: 1.0847
Epoch [210/1000] - Training Loss: 1.0436
Epoch [210/1000] - Validation Loss: 1.0860
Epoch [211/1000] - Training Loss: 1.0436
Epoch [211/1000] - Validation Loss: 1.0851
Epoch [212/1000] - Training Loss: 1.0436
Epoch [212/1000] - Validation Loss: 1.0832
Epoch [213/1000] - Training Loss: 1.0436
Epoch [213/1000] - Validation Loss: 1.0833
Epoch [214/1000] - Training Loss: 1.0436
Epoch [214/1000] - Validation Loss: 1.0842
Epoch [215/1000] - Training Loss: 1.0436
Epoch [215/1000] - Validation Loss: 1.0850
Epoch [216/1000] - Training Loss: 1.0436
Epoch [216/1000] - Validation Loss: 1.0849
Epoch [217/1000] - Training Loss: 1.0436
Epoch [217/1000] - Validation Loss: 1.0842
Epoch [218/1000] - Training Loss: 1.0436
Epoch [218/1000] - Validation Loss: 1.0841
Epoch [219/1000] - Training Loss: 1.0436
Epoch [219/1000] - Validation Loss: 1.0838
Epoch [220/1000] - Training Loss: 1.0436
Epoch [220/1000] - Validation Loss:

Epoch [308/1000] - Training Loss: 1.0436
Epoch [308/1000] - Validation Loss: 1.0856
Epoch [309/1000] - Training Loss: 1.0436
Epoch [309/1000] - Validation Loss: 1.0854
Epoch [310/1000] - Training Loss: 1.0436
Epoch [310/1000] - Validation Loss: 1.0848
Epoch [311/1000] - Training Loss: 1.0436
Epoch [311/1000] - Validation Loss: 1.0840
Epoch [312/1000] - Training Loss: 1.0436
Epoch [312/1000] - Validation Loss: 1.0849
Epoch [313/1000] - Training Loss: 1.0436
Epoch [313/1000] - Validation Loss: 1.0831
Epoch [314/1000] - Training Loss: 1.0436
Epoch [314/1000] - Validation Loss: 1.0831
Epoch [315/1000] - Training Loss: 1.0436
Epoch [315/1000] - Validation Loss: 1.0834
Epoch [316/1000] - Training Loss: 1.0436
Epoch [316/1000] - Validation Loss: 1.0839
Epoch [317/1000] - Training Loss: 1.0436
Epoch [317/1000] - Validation Loss: 1.0828
Epoch [318/1000] - Training Loss: 1.0436
Epoch [318/1000] - Validation Loss: 1.0828
Epoch [319/1000] - Training Loss: 1.0436
Epoch [319/1000] - Validation Loss:

Epoch [406/1000] - Training Loss: 1.0436
Epoch [406/1000] - Validation Loss: 1.0826
Epoch [407/1000] - Training Loss: 1.0436
Epoch [407/1000] - Validation Loss: 1.0808
Epoch [408/1000] - Training Loss: 1.0436
Epoch [408/1000] - Validation Loss: 1.0800
Epoch [409/1000] - Training Loss: 1.0436
Epoch [409/1000] - Validation Loss: 1.0848
Epoch [410/1000] - Training Loss: 1.0436
Epoch [410/1000] - Validation Loss: 1.0846
Epoch [411/1000] - Training Loss: 1.0436
Epoch [411/1000] - Validation Loss: 1.0839
Epoch [412/1000] - Training Loss: 1.0436
Epoch [412/1000] - Validation Loss: 1.0827
Epoch [413/1000] - Training Loss: 1.0436
Epoch [413/1000] - Validation Loss: 1.0827
Epoch [414/1000] - Training Loss: 1.0436
Epoch [414/1000] - Validation Loss: 1.0845
Epoch [415/1000] - Training Loss: 1.0436
Epoch [415/1000] - Validation Loss: 1.0848
Epoch [416/1000] - Training Loss: 1.0436
Epoch [416/1000] - Validation Loss: 1.0816
Epoch [417/1000] - Training Loss: 1.0436
Epoch [417/1000] - Validation Loss:

...

Epoch [603/1000] - Training Loss: 1.0436
Epoch [603/1000] - Validation Loss: 1.0844
Epoch [604/1000] - Training Loss: 1.0436
Epoch [604/1000] - Validation Loss: 1.0848
Epoch [605/1000] - Training Loss: 1.0436
Epoch [605/1000] - Validation Loss: 1.0855
Epoch [606/1000] - Training Loss: 1.0436
Epoch [606/1000] - Validation Loss: 1.0864
Epoch [607/1000] - Training Loss: 1.0436
Epoch [607/1000] - Validation Loss: 1.0864
Epoch [608/1000] - Training Loss: 1.0436
Epoch [608/1000] - Validation Loss: 1.0866
Epoch [609/1000] - Training Loss: 1.0436
Epoch [609/1000] - Validation Loss: 1.0867
Epoch [610/1000] - Training Loss: 1.0436
Epoch [610/1000] - Validation Loss: 1.0866
Epoch [611/1000] - Training Loss: 1.0436
Epoch [611/1000] - Validation Loss: 1.0869
Epoch [612/1000] - Training Loss: 1.0436
Epoch [612/1000] - Validation Loss: 1.0868
Epoch [613/1000] - Training Loss: 1.0436
Epoch [613/1000] - Validation Loss: 1.0868
Epoch [614/1000] - Training Loss: 1.0436
Epoch [614/1000] - Validation Loss:

Epoch [701/1000] - Training Loss: 1.0436
Epoch [701/1000] - Validation Loss: 1.0836
Epoch [702/1000] - Training Loss: 1.0436
Epoch [702/1000] - Validation Loss: 1.0835
Epoch [703/1000] - Training Loss: 1.0436
Epoch [703/1000] - Validation Loss: 1.0835
Epoch [704/1000] - Training Loss: 1.0436
Epoch [704/1000] - Validation Loss: 1.0834
Epoch [705/1000] - Training Loss: 1.0436
Epoch [705/1000] - Validation Loss: 1.0834
Epoch [706/1000] - Training Loss: 1.0436
Epoch [706/1000] - Validation Loss: 1.0832
Epoch [707/1000] - Training Loss: 1.0436
Epoch [707/1000] - Validation Loss: 1.0831
Epoch [708/1000] - Training Loss: 1.0436
Epoch [708/1000] - Validation Loss: 1.0831
Epoch [709/1000] - Training Loss: 1.0436
Epoch [709/1000] - Validation Loss: 1.0831
Epoch [710/1000] - Training Loss: 1.0436
Epoch [710/1000] - Validation Loss: 1.0830
Epoch [711/1000] - Training Loss: 1.0436
Epoch [711/1000] - Validation Loss: 1.0828
Epoch [712/1000] - Training Loss: 1.0436
Epoch [712/1000] - Validation Loss:

Epoch [802/1000] - Training Loss: 1.0436
Epoch [802/1000] - Validation Loss: 1.0832
Epoch [803/1000] - Training Loss: 1.0436
Epoch [803/1000] - Validation Loss: 1.0838
Epoch [804/1000] - Training Loss: 1.0436
Epoch [804/1000] - Validation Loss: 1.0839
Epoch [805/1000] - Training Loss: 1.0436
Epoch [805/1000] - Validation Loss: 1.0837
Epoch [806/1000] - Training Loss: 1.0436
Epoch [806/1000] - Validation Loss: 1.0828
Epoch [807/1000] - Training Loss: 1.0436
Epoch [807/1000] - Validation Loss: 1.0822
Epoch [808/1000] - Training Loss: 1.0436
Epoch [808/1000] - Validation Loss: 1.0822
Epoch [809/1000] - Training Loss: 1.0436
Epoch [809/1000] - Validation Loss: 1.0824
Epoch [810/1000] - Training Loss: 1.0436
Epoch [810/1000] - Validation Loss: 1.0828
Epoch [811/1000] - Training Loss: 1.0436
Epoch [811/1000] - Validation Loss: 1.0816
Epoch [812/1000] - Training Loss: 1.0436
Epoch [812/1000] - Validation Loss: 1.0817
Epoch [813/1000] - Training Loss: 1.0436
Epoch [813/1000] - Validation Loss:

Epoch [900/1000] - Training Loss: 1.0436
Epoch [900/1000] - Validation Loss: 1.0852
Epoch [901/1000] - Training Loss: 1.0436
Epoch [901/1000] - Validation Loss: 1.0848
Epoch [902/1000] - Training Loss: 1.0436
Epoch [902/1000] - Validation Loss: 1.0846
Epoch [903/1000] - Training Loss: 1.0436
Epoch [903/1000] - Validation Loss: 1.0843
Epoch [904/1000] - Training Loss: 1.0436
Epoch [904/1000] - Validation Loss: 1.0829
Epoch [905/1000] - Training Loss: 1.0436
Epoch [905/1000] - Validation Loss: 1.0813
Epoch [906/1000] - Training Loss: 1.0436
Epoch [906/1000] - Validation Loss: 1.0819
Epoch [907/1000] - Training Loss: 1.0436
Epoch [907/1000] - Validation Loss: 1.0878
Epoch [908/1000] - Training Loss: 1.0436
Epoch [908/1000] - Validation Loss: 1.0885
Epoch [909/1000] - Training Loss: 1.0436
Epoch [909/1000] - Validation Loss: 1.0882
Epoch [910/1000] - Training Loss: 1.0436
Epoch [910/1000] - Validation Loss: 1.0882
Epoch [911/1000] - Training Loss: 1.0436
Epoch [911/1000] - Validation Loss:

Epoch [998/1000] - Training Loss: 1.0436
Epoch [998/1000] - Validation Loss: 1.0855
Epoch [999/1000] - Training Loss: 1.0436
Epoch [999/1000] - Validation Loss: 1.0862
Epoch [1000/1000] - Training Loss: 1.0436
Epoch [1000/1000] - Validation Loss: 1.0867


In [21]:
writer.close()

In [29]:
model_path = "model.pth"
torch.save(model, model_path)

# Log the model as an artifact
mlflow.pytorch.log_model(model, artifact_path="model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/vboxuser/mlprojects/sample/env/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
mlflow.log_artifact("model.json", artifact_path="model_architecture")
mlflow.end_run()

In [22]:
!tensorboard --logdir='./runs' --port=6006

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Traceback (most recent call last):
  File "/home/vboxuser/mlprojects/sample/env/bin/tensorboard", line 6, in <module>
    from tensorboard.main import run_main
  File "/home/vboxuser/mlprojects/sample/env/lib/python3.9/site-packages/tensorboard/main.py", line 43, in <module>
    from tensorboard import default
  File "/home/vboxuser/mlprojects/sample/env/lib/python3.9/site-packages/tensorboard/default.py", line 36, in <module>
    from tensorboard.plugins.audio import audio_plugin
  File "/home/vboxuser/mlprojects/sample/env/lib/python3.9/site-packages/tensorboard/plugins/audio/audio_plugin.py", line 22, in <module>
    from werkzeug import wrappers
  File "/home/vbox

In [ ]:
# Inside the training loop
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt", max_length=max_length)
        labels = batch['label']
        
        with torch.no_grad():
            bert_outputs = pretrained_model(**inputs).last_hidden_state
        
        cls_token_representation = bert_outputs[:, 0, :]
        logits = custom_classifier(cls_token_representation)  # Pass [CLS] token's representation
        
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class TwoLayerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TwoLayerClassifier, self).__init__()
        self.hidden_layer = nn.Linear(input_dim, hidden_dim)
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        hidden_output = torch.relu(self.hidden_layer(x))
        output = self.output_layer(hidden_output)
        output_probs = self.softmax(output)
        return output_probs

# Define dimensions
input_dim = 512
hidden_dim = 256
output_dim = 5

# Create the model
model = TwoLayerClassifier(input_dim, hidden_dim, output_dim)

# Define example data and labels
data = torch.randn(100, input_dim)  # 100 samples
labels = torch.randint(0, output_dim, (100,))  # 100 labels

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

print("Training finished!")

# Evaluate on new data
new_data = torch.randn(20, input_dim)  # New 20 samples
with torch.no_grad():
    predictions = model(new_data)
    predicted_labels = torch.argmax(predictions, dim=1)
    print("Predicted labels:", predicted_labels)
